# Model Retraining and Monitoring - Bridge Failure Prediction

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import joblib
import matplotlib.pyplot as plt
from src.models.train import drift_detection

## Load Current and New Data

In [2]:
df_current = pd.read_csv("../data/processed/features.csv")
df_new = pd.read_csv("../data/processed/features_new.csv")

## Drift Monitoring

In [3]:
features = [c for c in df_current.columns if c not in ['structure_id', 'failure_within_1yr']]
drift = drift_detection(df_new, df_current, features)
import pandas as pd
drift_df = pd.DataFrame.from_dict(drift, orient='index')
drift_df.sort_values('p_value').head()

## Retrain Model if Significant Drift Detected

In [4]:
if (drift_df['p_value'] < 0.05).any():
    print("Significant drift detected! Retraining model...")
    X = df_new.drop(['failure_within_1yr', 'structure_id'], axis=1)
    y = df_new['failure_within_1yr']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    model = RandomForestClassifier(n_estimators=100, max_depth=7, random_state=42)
    model.fit(X_train, y_train)
    joblib.dump(model, '../models/trained/model_retrained.joblib')
else:
    print("No significant drift detected. No retraining needed.")

## Evaluate Retrained Model

In [5]:
model = joblib.load('../models/trained/model_retrained.joblib')
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:,1]
print('ROC AUC:', roc_auc_score(y_test, y_proba))

## Plot Feature Importances (Retrained)

In [6]:
importances = model.feature_importances_
features_names = X_test.columns
pd.Series(importances, index=features_names).sort_values(ascending=False).plot(kind='bar', figsize=(12,4))
plt.title('Retrained Model Feature Importance')
plt.tight_layout()
plt.show()